# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

In [5]:


SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
count = 0
while True:
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    pieces = soup.find('div',{'class':'part_pictxt_3 lazyload'}).find_all('div',{'class':'piece clearfix'})
    if len(pieces) != count:
        count = len(pieces)
        print('網頁更新中...')
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight),10000;")
    else:
        print('到達頁面底端')
        break

網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新台幣開盤貶值0.5分　以30.22元兌1美元開出

▲ 新台幣兌美元今天開盤貶值0.5分。（圖／路透社）
記者李瑞瑾／台北報導
新台幣匯率維持高檔震盪，台北外匯市場新台幣兌美元匯率今（24）天開盤貶值0.5分、以30.22兌1美元開出。
新台幣兌美元昨（23）日收盤收在30.215兌1美元；今天開盤以30.22元開出、貶值0.5分。
開盤後新台幣轉為升值，截至上午9點13分，新台幣為30.176兌1美元，升值3.9分。
----------------------------------------------------------------------

[1] 美股續創新高！台股未受激勵　開盤小漲1點後於平盤震盪

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
中美貿易協議明朗化、雙方對彼此釋出善意，美股四大指數全面走揚，續創歷史新高。台股今（24）日開盤未受激勵，僅小漲1點、以12,023點開出。
台股開盤後持續在平盤上下震盪，截至上午9點05分，指數下跌4點為12,017點。
電子三雄今日全部開在平盤，台積電（2330）以334元開出；鴻海（2317）開在91.5元；股王大立光（3008）以4870元開出。
在美國總統川普重申第一屆段協議將很快簽署完成之後，中國大陸也對美方釋出利多，宣布從明年元旦起降低美國商品進口關稅，並低於最惠國稅率，激勵美股四大指數全面走揚，再創歷史新高紀錄。
美股道瓊指數上漲96.44點，或0.34%，收28551.53點；那斯達克指數上漲20.69點，或0.23%，收8945.65點；標普 500 指數上漲2.79點，或0.09%，收3224.01點；費城半導體指數上漲3.25點，或0.18%，收1857.12點。
----------------------------------------------------------------------

[2] 新加坡富商「台北東區擺攤遇貴人」攢出百億身家　為SOGO砲打經濟部

圖、文／鏡週刊
▲新加坡麵包大亨郭明忠為投資SOGO，砲打經濟部，並準備提起國際仲裁。（圖／鏡週刊提供，下同）
「遠東郭明宗（時任遠紡

▲星展銀預估，G3國家央行明年將暫停降息，但仍維持貨幣寬鬆低利環境，股市可望受惠。（示意圖／CFP）
記者紀佳妘／台北報導
星展銀行（DBS）今（23）日發布2020年第1季投資展望，在公債殖利率曲線過度平坦，且利率已處於極低的水準下，預估明年G3（美國、日本、歐元區）國家央行將維持寬鬆貨幣政策，並暫停進一步降息。
星展銀行發布2020年第1季投資展望報告指出，明年的美國經濟將受到具有韌性的國內消費和謹慎的投資成長共2個因素的多空拉鋸，歐元區的經濟成長仍將處於較為疲弱的狀態。不過，中美貿易戰在2020年依然會是市場擔憂的事件之一，同時也需留意英國脫歐和香港社會運動事件的發展。
報告顯示，儘管明年通膨率顯著上揚的機會不高，但可望觸底反彈，且從食品和基礎金屬價格的走勢來看，不會出現通貨緊縮。此外，由於寬鬆貨幣政策的效果有限，各國政策重點將著重在財政政策，市場關注焦點將轉至德國。
展望金融市場，星展銀認為，投資人的股市部位仍偏低，公債殖利率上揚的環境有利股市在2020年的表現，加上美國企業獲利表現較良好，持續相對看好美股。不過，即使貿易戰所引起的經濟衰退擔憂，2019年全球股市漲幅仍接近23%，美股也第11年走揚，為有史以來最長的牛市，同時也是最不受市場青睞的牛市，因市場對股市能否持續走揚存疑，因此投資人布局股市的比重偏低。
星展銀進一步指出，若市場維持具有韌性的表現，持有大量現金的投資組合可能會被迫重新投入市場，基於害怕錯過（FOMO）、投資人別無選擇（TINA）的心理，預期在極低利率和債券殖利率的環境下，投資人只能透過布局股票市場獲取較佳的報酬；另外考慮到違約率與到期殖利率，BBB／BB評級的債券較具有投資價值，相對看好新興市場債，預期信貸利差將會收緊。
至於外匯市場，星展銀表示，貨幣政策走向的分歧，將持續支撐美元指數，由於日圓會有避險相關的需求，需要寬鬆貨幣政策才會使日圓的表現較為弱勢；另在利率市場，則預估G3國家公債殖利率曲線將轉趨陡峭，其中短天期利率將持穩，而經濟成長動能的改善將溫和推升長天期利率。
展望2020年，星展銀持續偏好以槓鈴式投資策略建構投資組合，同時布局創造收益的資產，以及受惠於數位化潮流、人口老化以及大陸中產階級成長等題材的股市。此外，受惠於直播的興盛，電子競技相關產業將有顯著的成長，而數位化的潮流將帶動全球半導體的需求。
-----------

▲Google新任執行長皮查伊（Sundar Pichai）。（圖／達志影像）
記者謝仁傑／綜合報導
現任Google執行長皮查伊（Sundar Pichai）成為全球薪資最高的高階管理人之一。皮查伊在過去的幾年中擔任Google的CEO，年薪為65萬美元（約新台幣1964萬元）；然而根據美國證券交易委員會（SEC）提交的文件顯示，2020年皮查伊的年薪將增至200萬美元（約新台幣6042萬元），比原來年薪增加了2倍左右。
另外，他還有機會領取高額股票分紅，如果達標，未來幾年最高可領走價值2.4億美元（約新台幣72.5億元）的分紅。
皮查伊在兼任Google和母公司Alphabet的CEO後，職責擴大，像其他高階主管一樣，皮查伊在滿足公司特定績效指標或績效股單位（PSU）後，還可領股票報酬。
另外，如果Alphabet股價表現超越同期的S&P 100指數，每年約可多拿價值4500萬美元（約新台幣13.6億元）的股票，兩年合計9千萬美元。
換句話說，未來三年，Pichai如果業績達標，獲得的分紅總值將達2.4億美元（約新台幣72.5億元）。
Google聯合創辦人佩奇（Larry Page）與布林（Sergey Brin）各持有Alphabet約5.8%、5.6%股份，新的接班人皮查伊持股約0.1%，這意味著新任執行長仍會受到董事會包括創始人的掌控。加上近期美國各州對於反壟斷及隱私權的調查，以及歐洲最近對於Google獨佔市場的判決，皮查伊這個執行長雖然領得多，也確實不好當。
